In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import requests #type: ignore
import base64
import pandas as pd
import sys;sys.path.append('../../')
from listener_effort_api.utils import get_logger
logger = get_logger()

def load_wav_file(file_path):
    with open(file_path, 'rb') as f:
        return f.read()

In [3]:
# We choose study from which to get the data
study = 'Radcliff' # 'Radcliff' # 'Speech_study' # 'Prilenia'

In [4]:
df = pd.read_csv(f'/home/ubuntu/git/LEPM/datasets/{study}_dataset_with_features_and_le.csv')
df = df[df.task_name.str.count('Rotating')>0].reset_index(drop=True)
df.head(3)

,user_id,session_id,wav_path,prompt_text,task_name,pac,whisper_confidence_base,whisper_confidence_large_v2,speaking_rate_base,speaking_rate_large_v2,...,pitch_mean,pitch_std,cpp_1,wiener_entropy,drop_from_notes,is_unable_to_rate,effort_mean,effort_list,effort_mean_session,effort_list_session
0,03cd72ae-fbf6-484b-9d86-09f7230726bf,20230927-162950-A91472,/data-output/radcliff_data/2024-05-03/raw/aura...,The supermarket chain shut down because of poo...,12RotatingSentencesFixed1,True,NaN,NaN,NaN,NaN,...,186.896576,29.500917,21.897563,0.038786,NaN,NaN,NaN,NaN,12.777778,[ 8 6 23 11 6 5 18 15 23]
1,03cd72ae-fbf6-484b-9d86-09f7230726bf,20230927-162950-A91472,/data-output/radcliff_data/2024-05-03/raw/aura...,Much more money must be donated to make this d...,12RotatingSentencesFixed2,True,NaN,NaN,NaN,NaN,...,183.364282,22.636446,22.139333,0.020759,NaN,NaN,NaN,NaN,12.777778,[ 8 6 23 11 6 5 18 15 23]
2,03cd72ae-fbf6-484b-9d86-09f7230726bf,20230927-162950-A91472,/data-output/radcliff_data/2024-05-03/raw/aura...,"In this famous coffee shop, they serve the bes...",12RotatingSentencesFixed3,True,NaN,NaN,NaN,NaN,...,187.565376,31.606340,21.843565,0.019996,NaN,NaN,NaN,NaN,12.777778,[ 8 6 23 11 6 5 18 15 23]


In [119]:
import base64
import requests
import os
from dotenv import load_dotenv

def load_wav_file(file_path):
    with open(file_path, 'rb') as f:
        return f.read()

payload = {
    "input": [
        {
            "audios": [
                {
                   "wav": base64.b64encode(load_wav_file('/data-output/radcliff_data/2024-05-03/raw/aural/03cd72ae-fbf6-484b-9d86-09f7230726bf/20230927-162950-A91472/19.wav')).decode(),
                    "transcript": "The forest near my grandpa's cabin is said to contain mythical creatures."
                },
                {
                    "wav": base64.b64encode(load_wav_file('/data-output/radcliff_data/2024-05-03/raw/aural/03cd72ae-fbf6-484b-9d86-09f7230726bf/20230927-162950-A91472/20.wav')).decode(),
                    "transcript": None
                },
                {
                    "wav": base64.b64encode(load_wav_file('/data-output/radcliff_data/2024-05-03/raw/aural/03cd72ae-fbf6-484b-9d86-09f7230726bf/20230927-162950-A91472/21.wav')).decode(),
                    "transcript": None
                }
            ]
        },
        {
            "audios": [
                {
                   "wav": base64.b64encode(load_wav_file('/data-output/radcliff_data/2024-05-03/raw/aural/8430613e-9cfe-464c-b657-f24ea1e3b5f1/20230922-222930-A73246/19.wav')).decode(),
                    "transcript": 'Why did the critics rate this book so favorably?'
                },
                {
                    "wav": base64.b64encode(load_wav_file('/data-output/radcliff_data/2024-05-03/raw/aural/8430613e-9cfe-464c-b657-f24ea1e3b5f1/20230922-222930-A73246/20.wav')).decode(),
                    "transcript": 'Many of his songs are the products of collaborations with other musicians.'
                },
            
            ]
        }
    ]
}

load_dotenv()
token = os.getenv("EALS_LE_API_KEY")
headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {token}"
}

response = requests.post('http://localhost:8000/v1/listener-effort', json=payload, headers=headers)
print(response.status_code)
response.json()

200


{'status': 'ok',
 'result': [{'status': 'ok',
   'listener_effort': 0.0,
   'listener_effort_stddev': 5.326153429734596,
   'audio_results': [{'status': 'ok', 'listener_effort': 12.440047681324618},
    {'status': 'ok', 'listener_effort': 2.8156873602877965},
    {'status': 'ok', 'listener_effort': 0.0}]},
  {'status': 'ok',
   'listener_effort': 80.61688118273041,
   'listener_effort_stddev': 13.65227293932476,
   'audio_results': [{'status': 'ok', 'listener_effort': 94.26915412205517},
    {'status': 'ok', 'listener_effort': 66.96460824340565}]}]}

In [13]:
from listener_effort_api.lepm import batch_predict_le
from listener_effort_api.items import PredictRequest

req = PredictRequest.from_dict(payload)

# Decode the base64-encoded audio data in the request
for session in req.input:
    for audio in session.audios:
        audio.wav = base64.b64decode(audio.wav)
res = batch_predict_le(req)
res.dict()

/tmp/ipykernel_152197/2230580957.py:11: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.11/migration/
  res.dict()


{'status': 'error',
 'result': [{'status': 'mock error occurred',
   'listener_effort': 35.43143530053009,
   'listener_effort_stddev': 54.42652519405584,
   'audio_results': [{'status': 'ok', 'listener_effort': 17.504204925418243},
    {'status': 'mock error occurred', 'listener_effort': nan},
    {'status': 'ok', 'listener_effort': 79.99339688621718}]},
  {'status': 'ok',
   'listener_effort': 17.049522391237847,
   'listener_effort_stddev': 30.093137461061083,
   'audio_results': [{'status': 'ok', 'listener_effort': 25.379743666589462},
    {'status': 'ok', 'listener_effort': 96.64148288844913}]}]}